In [1]:
%load_ext autoreload
%autoreload 2

# Install (not use google storage bucket)

In [2]:
# Run once (not use google storage bucket)
# !pip install transformers
# !pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 23.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 32.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 20.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.7/749.7 KB 13.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a607e30776ba4310207d454c69dd63148d28567a0d31cc1eeba31e71e1600d4b
  Stored in directory: /home/jupyter/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:0000:010:01


In [4]:
!which python

/opt/conda/bin/python


## Using gcsfuse + google storage bucket (not used on Vertex notebook)



In [2]:
%%writefile /ideta-ml-thi-thivietnam.json
{
  "key": "keys"

%env GOOGLE_APPLICATION_CREDENTIALS=/ideta-ml-thi-thivietnam.json

In [ ]:
!printenv GOOGLE_APPLICATION_CREDENTIALS

To install once the packages!

In [ ]:
# Install gcsfuse + google storage bucket
# Use once (for each google acc)
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  29500      0 --:--:-- --:--:-- --:--:-- 29500
OK
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 52 not upgraded.
Need to get 12.0 MB of archives.
After this operation, 28.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.40.0_amd64.deb ...
Unpacking gcsfuse (0.40.0) ...
Setting up gcsfuse (0.40.0) ...


In [ ]:
!mkdir googleBucketFolder
!gcsfuse --implicit-dirs colab-connect-bucket googleBucketFolder

2022/04/21 07:36:02.337730 Start gcsfuse/0.40.0 (Go version go1.17.6) for app "" using mount point: /content/googleBucketFolder
2022/04/21 07:36:02.350298 Opening GCS connection...
2022/04/21 07:36:02.559602 Mounting file system "colab-connect-bucket"...
2022/04/21 07:36:02.594314 File system has been successfully mounted.


In [ ]:
import sys
nb_path = 'googleBucketFolder'

In [ ]:
sys.path.insert(0, nb_path)

In [ ]:
# Do this just once
!pip install --target=$nb_path transformers

In [ ]:
!pip install --target=$nb_path sentencepiece

     |████████████████████████████████| 1.2 MB 15.8 MB/s eta 0:00:01


In [ ]:
!pip show transformers

Name: transformers
Version: 4.18.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: tqdm, huggingface-hub, regex, tokenizers, importlib-metadata, packaging, sacremoses, requests, numpy, filelock, pyyaml
Required-by: 


# Check GPU 

In [2]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Variables

In [5]:
# !mkdir ../../model
!ls ../../../

google-cloud-sdk  ideta-logos  imported  model	src  tutorials


In [6]:
BUCKET_NAME = 'gs://ideta-sentiment-analysis/'
PROJECT_ID = 'ideta-ml-thi'
REGION = 'europe-west1'
data_loc = "../../../model"

# Link with google storage

In [7]:
# Finally, validate access to your Cloud Storage bucket by examining its contents
! gsutil ls -al

gs://artifacts.ideta-ml-thi.appspot.com/
gs://ideta-sentiment-analysis/
gs://torch-text-class-testing/


# Model `xlm-roberta-large-xnli` (multi languages)

Model: [xlm-roberta-large-xnli](https://huggingface.co/joeddav/xlm-roberta-large-xnli?candidateLabels=negative%2Cpositive%2Cneutral&multiClass=false&text=It%27s+neither+bad+nor+good+to+me)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")

## From and to Torch format

In [8]:
pt_model_dir = data_loc + '/pt-xlm-roberta-large-xnli'
pt_model_dir

'../../../model/pt-xlm-roberta-large-xnli'

Don't run below if you have already saved the model.

In [11]:
pt_model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Save to disk

In [13]:
tokenizer.save_pretrained(pt_model_dir)

('../../../model/pt-xlm-roberta-large-xnli/tokenizer_config.json',
 '../../../model/pt-xlm-roberta-large-xnli/special_tokens_map.json',
 '../../../model/pt-xlm-roberta-large-xnli/sentencepiece.bpe.model',
 '../../../model/pt-xlm-roberta-large-xnli/added_tokens.json',
 '../../../model/pt-xlm-roberta-large-xnli/tokenizer.json')

In [14]:
pt_model.save_pretrained(pt_model_dir)

### Load from saved

In [9]:
pt_tokenizer = AutoTokenizer.from_pretrained(pt_model_dir)
pt_model = AutoModelForSequenceClassification.from_pretrained(pt_model_dir)

### Encode and decode with `tokenizer`

We try to encode and decode string with tokenizer.

In [10]:
inputs = pt_tokenizer("I\'m really disapointed with what are you saying and your service, give my money back!!!!", return_tensors="pt")

In [11]:
inputs

{'input_ids': tensor([[    0,    87,    25,    39,  6183,  6392, 38496,   297,   678,  2367,
           621,   398, 54433,   136,   935,  4516,     4,  8337,   759, 17265,
          4420, 11305,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [13]:
pt_tokenizer.decode(inputs["input_ids"])

TypeError: 'list' object cannot be interpreted as an integer

In [14]:
inputs_2 = pt_tokenizer("I\'m really disapointed with what are you saying and your service, give my money back!!!!")
inputs_2

{'input_ids': [0, 87, 25, 39, 6183, 6392, 38496, 297, 678, 2367, 621, 398, 54433, 136, 935, 4516, 4, 8337, 759, 17265, 4420, 11305, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
pt_tokenizer.decode(inputs_2["input_ids"])

"<s> I'm really disapointed with what are you saying and your service, give my money back!!!!</s>"

## From and to TF format

In [1]:
saved_loc = '/hf-xlm-roberta-large-xnli'

In [7]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

2022-05-04 11:25:53.977736: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [11]:
tf_model.save_pretrained(data_loc + saved_loc)

In [13]:
tokenizer.save_pretrained(data_loc + saved_loc)

('../../model/tokenizer_config.json',
 '../../model/special_tokens_map.json',
 '../../model/sentencepiece.bpe.model',
 '../../model/added_tokens.json',
 '../../model/tokenizer.json')

In [11]:
! pip show torch

Name: torch
Version: 1.11.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.7/site-packages
Requires: typing-extensions
Required-by: 


### Reused download model

https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModel

In [8]:
data_loc + saved_loc

'../../../model/hf-xlm-roberta-large-xnli'

Get the tokenizer,

In [9]:
tokenizer = AutoTokenizer.from_pretrained(data_loc + saved_loc)

Import model using `TFAutoModel`

In [ ]:
from transformers import TFAutoModel, AutoTokenizer
tf_model = TFAutoModel.from_pretrained(data_loc + saved_loc)

2022-05-11 20:42:38.023642: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


__Import using `TFAutoModelForSequenceClassification` instead of `TFAutoModel`??__

In [15]:
from transformers import TFAutoModelForSequenceClassification

In [16]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained(data_loc + saved_loc)

2022-05-11 21:30:47.894392: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at ../../../model/hf-xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [77]:
tf_model

In [79]:
num_labels = len(tf_model.config.id2label)
num_labels

3

https://huggingface.co/docs/transformers/model_doc/xlm-roberta#transformers.TFXLMRobertaForSequenceClassification

In [57]:
tf_model.config.id2label[0], tf_model.config.id2label[1], tf_model.config.id2label[2]

('contradiction', 'neutral', 'entailment')

### Using model outputs

https://huggingface.co/docs/transformers/main/en/main_classes/output

In [48]:
inputs = tokenizer("I\'m really disapointed with what are you saying and your service, give my money back!!!!", return_tensors="tf")

In [39]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[    0,    87,    25,    39,  6183,  6392, 38496,   297,   678,
         2367,   621,   398, 54433,   136,   935,  4516,     4,  8337,
          759, 17265,  4420, 11305,     2]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1]], dtype=int32)>}

In [36]:
tf_model

https://huggingface.co/docs/transformers/main/en/model_doc/xlm-roberta#transformers.TFXLMRobertaForSequenceClassification

In [38]:
output = tf_model(**inputs)

In [35]:
output

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-2.586414 ,  2.921955 , -0.8035885]], dtype=float32)>, hidden_states=None, attentions=None)

https://huggingface.co/docs/transformers/main/en/main_classes/output#transformers.modeling_tf_outputs.TFSequenceClassifierOutput

In [53]:
logits = tf_model(**inputs).logits
print('logits: ', logits)
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
print('predicted_class_id: ', predicted_class_id)
tf_model.config.id2label[predicted_class_id]

logits:  tf.Tensor([[-2.586414   2.921955  -0.8035885]], shape=(1, 3), dtype=float32)
predicted_class_id:  1


'neutral'

In [56]:
tf_model.config.id2label[0], tf_model.config.id2label[1], tf_model.config.id2label[2]

('contradiction', 'neutral', 'entailment')

In [75]:
output_2 = tf_model(inputs["input_ids"], return_dict=True)
output_2

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-2.586414 ,  2.921955 , -0.8035885]], dtype=float32)>, hidden_states=None, attentions=None)

❓ __Why get 3 labels `('contradiction', 'neutral', 'entailment')`by default???__ => Try different tokenizer params?

https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#tokenizer

In [76]:
inputs = tokenizer(
    "I\'m really disapointed with what are you saying and your service, give my money back!!!!",
    return_tensors="tf",
    model_input_names=['positive', 'negative', 'neutral'])

TypeError: _batch_encode_plus() got an unexpected keyword argument 'model_input_names'

### Using `pipeline`

__Task with `pipeline`__: https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline.task

In [58]:
from transformers import pipeline

In [59]:
pipe = pipeline(task='zero-shot-classification', model=data_loc+saved_loc, tokenizer=data_loc+saved_loc)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at ../../../model/hf-xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


And then predict,

In [64]:
# english -> negative
pipe(
    'I\'m really disapointed with what are you saying and your service, give my money back!!!!',
    candidate_labels=["positive", "negative", "neutral"],
)

{'sequence': "I'm really disapointed with what are you saying and your service, give my money back!!!!",
 'labels': ['negative', 'neutral', 'positive'],
 'scores': [0.9862303137779236, 0.009490575641393661, 0.004279122222214937]}

In [65]:
# english -> neutral
pipe(
    'It\'s neither bad nor good to me',
    candidate_labels=["positive", "negative", "neutral"],
)

{'sequence': "It's neither bad nor good to me",
 'labels': ['neutral', 'positive', 'negative'],
 'scores': [0.639499843120575, 0.3461650609970093, 0.014335119165480137]}

In [66]:
# vietnamese -> positive
pipe(
    'Tôi rất thích bài viết này của bạn',
    candidate_labels=["tích cực", "tiêu cực", "bình thường"],
)

{'sequence': 'Tôi rất thích bài viết này của bạn',
 'labels': ['tích cực', 'tiêu cực', 'bình thường'],
 'scores': [0.950676679611206, 0.027289491146802902, 0.02203379012644291]}

In [71]:
# vietnamese -> negative
pipe(
    'Dịch vụ rất kém',
    candidate_labels=["tích cực", "tiêu cực", "bình thường"],
)

{'sequence': 'Dịch vụ rất kém',
 'labels': ['tiêu cực', 'tích cực', 'bình thường'],
 'scores': [0.9924571514129639, 0.004163913428783417, 0.003378975670784712]}

In [72]:
# french -> neutral
pipe(
    'Ce n\'est ni mauvais ni bon pour moi',
    candidate_labels=["positif", "negatif", "neutre"],
)

{'sequence': "Ce n'est ni mauvais ni bon pour moi",
 'labels': ['neutre', 'positif', 'negatif'],
 'scores': [0.7017592191696167, 0.28289851546287537, 0.015342272818088531]}

__We can use labels as a different language as the input__

In [80]:
pipe(
    'Dịch vụ rất kém',
    candidate_labels=["positive", "negative", "neutral"],
)

{'sequence': 'Dịch vụ rất kém',
 'labels': ['negative', 'neutral', 'positive'],
 'scores': [0.9832550287246704, 0.012725306674838066, 0.004019691608846188]}

In [81]:
pipe(
    'Ce n\'est ni mauvais ni bon pour moi',
    candidate_labels=["positive", "negative", "neutral"],
)

{'sequence': "Ce n'est ni mauvais ni bon pour moi",
 'labels': ['neutral', 'positive', 'negative'],
 'scores': [0.5339967608451843, 0.4472317099571228, 0.01877153478562832]}

__Seperated scores__ (each is between [0,1] and sum of them != 1)

In [83]:
pipe(
    'Ce n\'est ni mauvais ni bon pour moi',
    candidate_labels=["positive", "negative", "neutral"],
    multi_label=True
)

{'sequence': "Ce n'est ni mauvais ni bon pour moi",
 'labels': ['neutral', 'positive', 'negative'],
 'scores': [0.9716978073120117, 0.9530502557754517, 0.05425785854458809]}

In [84]:
pipe(
    'Ce n\'est ni mauvais ni bon pour moi',
    candidate_labels=["positif", "negatif", "neutre"],
    multi_label=True
)

{'sequence': "Ce n'est ni mauvais ni bon pour moi",
 'labels': ['neutre', 'positif', 'negatif'],
 'scores': [0.9908363223075867, 0.9490876793861389, 0.1009441390633583]}

## Convert to `SavedModel`

Types: https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts#tensorflow

- (idea -> not working) https://github.com/huggingface/transformers/issues/6864#issuecomment-690086932
- (idea -> works) https://towardsdatascience.com/use-pre-trained-huggingface-models-in-tensorflow-serving-d2761f7e69f6

In [23]:
MAX_SEQ_LEN = 100

callable = tf.function(tf_model.call)
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="input_ids"), tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="attention_mask")])
tf_model.save(data_loc + '/xlm-roberta-large-xnli', signatures=concrete_function)


INFO:tensorflow:Assets written to: ../../model/xlm-roberta-large-xnli/assets


INFO:tensorflow:Assets written to: ../../model/xlm-roberta-large-xnli/assets


Reload the saved model (format SavedModel) and then make an predict to make sure it works normally,

## Upload to bucket

Source: https://github.com/mrdbourke/cs329s-ml-deployment-tutorial/blob/main/model_training.ipynb

In [24]:
# from google.colab import auth
# auth.authenticate_user()

In [28]:
modelPath = data_loc + '/xlm-roberta-large-xnli'

In [30]:
! ls $modelPath

assets	keras_metadata.pb  saved_model.pb  variables


In [32]:
! gsutil cp -r $modelPath $BUCKET_NAME

Copying file://../../model/xlm-roberta-large-xnli/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://../../model/xlm-roberta-large-xnli/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://../../model/xlm-roberta-large-xnli/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://../../model/xlm-roberta-large-xnli/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help cr

## Try pipline "sentiment-analysis" => use default model

https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
clasifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
clasifier('we love you')

[{'label': 'POSITIVE', 'score': 0.9998704195022583}]

In [ ]:
clasifier('i hope we can overcome this situation')

[{'label': 'POSITIVE', 'score': 0.9984887838363647}]

In [ ]:
clasifier(
    'I will choose the second option.'
)

[{'label': 'NEGATIVE', 'score': 0.8387357592582703}]

## Try TFBert....SavedModel (archived)

Source: https://github.com/huggingface/transformers/issues/6864#issuecomment-690086932

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification

a = tf.constant([[1,2,3,4,5]])
b = tf.constant([[1,1,1,1,1]])
inp = {"input_ids": a, "attention_mask": b}
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
model._saved_model_inputs_spec = None
model._set_save_spec(inp)
tf.saved_model.save(model, "/tmp")
model = tf.keras.models.load_model("/tmp")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:tensorflow:Assets written to: /tmp/assets


INFO:tensorflow:Assets written to: /tmp/assets


ValueError: ignored

In [ ]:
tf.saved_model.load("/tmp")

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f7ad01e02d0>

## Download model and then re-use => for Torch (archived)

https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

https://huggingface.co/docs/transformers/installation#fetch-models-and-tokenizers-to-use-offline

In [ ]:
data_loc = "../data/transformers/distilbert-base-uncased-finetuned-sst-2-english"

In [ ]:
# Save to local for offline use
tokenizer.save_pretrained(data_loc)
model.save_pretrained(data_loc)

In [ ]:
# Use downloaded model
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(data_loc)
model = AutoModel.from_pretrained(data_loc)

Some weights of the model checkpoint at ../data/transformers/distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pipe = pipeline(task='sentiment-analysis', model=data_loc, tokenizer=data_loc)

In [ ]:
pipe('This movie is filmed in Vietnam')

[{'label': 'NEGATIVE', 'score': 0.9540639519691467}]

In [ ]:
pipe('This movie is filmed in Hong Kong')

[{'label': 'POSITIVE', 'score': 0.9831559658050537}]

## Multiple labels => facebook/bart-large-mnli (archived)

https://huggingface.co/facebook/bart-large-mnli

In [ ]:
# with multiple labels
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
classifier(
    'I\'m really disapointed with what are you saying and your service, give my money back!!!!',
    candidate_labels=["positive", "negative", "neutral"],
)

{'labels': ['negative', 'neutral', 'positive'],
 'scores': [0.9936130046844482, 0.004523514304310083, 0.001863523619249463],
 'sequence': "I'm really disapointed with what are you saying and your service, give my money back!!!!"}

In [ ]:
classifier(
    'I will choose the second option.',
    candidate_labels=["positive", "negative", "neutral"],
)

{'labels': ['positive', 'negative', 'neutral'],
 'scores': [0.3554832339286804, 0.34847933053970337, 0.296037495136261],
 'sequence': 'I will choose the second option.'}

In [ ]:
classifier(
    'You are so funny :D',
    candidate_labels=["positive", "negative", "neutral"],
)

{'labels': ['positive', 'negative', 'neutral'],
 'scores': [0.8636406660079956, 0.06884679198265076, 0.06751253455877304],
 'sequence': 'You are so funny :D'}

In [ ]:
classifier(
    'It\'s neither bad nor good to me',
    candidate_labels=["positive", "negative", "neutral"],
)

{'labels': ['neutral', 'positive', 'negative'],
 'scores': [0.8657446503639221, 0.12338893115520477, 0.010866399854421616],
 'sequence': "It's neither bad nor good to me"}